# Dependencies
The first cell installs NGSolve, which is the Finite Element Simulation (FEM) package we're using.

In [3]:
try:
    import ngsolve
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/ngsolve-install-real.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
    import ngsolve

--2024-11-04 22:37:57--  https://fem-on-colab.github.io/releases/ngsolve-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4198 (4.1K) [application/x-sh]
Saving to: ‘/tmp/ngsolve-install.sh’

/tmp/ngsolve-instal 100%[===================>]   4.10K  --.-KB/s    in 0s      

2024-11-04 22:37:57 (40.6 MB/s) - ‘/tmp/ngsolve-install.sh’ saved [4198/4198]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ NGSOLVE_INSTALLED=/usr/local/share/fem-on-colab/ngsolve.installed
+ [[ ! -f /usr/local/share/fem-on-colab/ngsolve.installed ]]
+ OCC_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/f895fa7b/releases/occ-install

Next, we check that the notebook is setup for rendering the simulations! This cell does nothing but checks.

In [5]:
try:
    import webgui_jupyter_widgets
    from packaging.version import parse
    assert parse(webgui_jupyter_widgets.__version__) >= parse("0.2.18")
    print('Everything good!')
except:
    print("\x1b[31mYou need to update webgui_jupyter_widgets by running: \x1b[0m\npython3 -m pip install --upgrade webgui_jupyter_widgets")

Everything good!


Finally, the last thing we do for setup is to import the ngsolve library so we're ready to use it.

In [6]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *   # Opencascade for geometry modeling

# The simulation

The first task is to setup a mesh that we can simulate our equations on.

## The geometry
We're going to look at conduction inside a circle (imagine a slice from a steel bar), with convection on the outside. So lets create a circular mesh.

In [7]:
# Make a radius 1 circle
geom = Circle((0,0), 1).Face()
geom.edges.name = "outeredge"
# Turn it into a 2D mesh, with triangles with a maximum edge length of 0.1
mesh = Mesh(OCCGeometry(geom, dim=2).GenerateMesh(maxh=0.1))
Draw(mesh)

## Derivation of the FEM Heat Equations

FEM simulations do not directly solve the heat equation (that's the realm of analytical solutions). Instead, FEM solves the so-called "weak form" of the heat equation, which is just a formal mathematical way of defining a mesh and (separately) functions that have some equivalence to the original equation.

For better notes on the mathematics behind weak forms, I personally found [this set of notes very useful in understanding the mathematics](http://hplgit.github.io/INF5620/doc/pub/H14/fem/sphinx/._main_fem012.html). While I found the [Doplhinx tutorials very useful for some gaps around boundary conditions](https://jsdokken.com/dolfinx-tutorial/chapter3/robin_neumann_dirichlet.html).


The equation we're going to solve here is the heat diffusion equation
$$
\rho\,C_p\frac{\partial T}{\partial t} = -\rho\,C_p\,\vec{v}
  \cdot\nabla\,T - \nabla\cdot\vec{q} - \vec{\tau}:\nabla\,\vec{v} -
  p\,\nabla\cdot\vec{v} +\sigma_{energy}
$$

which for stationary solids $\vec{v}=0$, becomes

$$
\rho\,C_p\frac{\partial T}{\partial t} = - \nabla\cdot\vec{q} +\sigma_{energy}
$$

Inserting fourier's law,
$$
\rho\,C_p\frac{\partial T}{\partial t} = \nabla\cdot k\,\nabla\,T +\sigma_{energy}
$$

So far we've been exact. To make this into a finite element problem where we solve an approximation, we define a set of trial functions $v$ we
"project" this equation onto. We are creating the so-called weak form of the
equation where it applies to a test function.

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega - \int_{\Omega} (\nabla\cdot k\nabla\vec{T})\,v\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega
$$

where the volume integrals are over the domain $\Omega$ (the circular mesh defined earlier).

This form is problematic, as we don't know how to add our particular (convection/Robin) boundary conditions, and we have a second order derivative in $T$ (which is more difficult in FEM). We'll use an identity to fix both of these problems.

### The key identity
The identity we need is based on [divergence theorem](https://en.wikipedia.org/wiki/Divergence_theorem#Mathematical_statement), which is,

$$ \int_\Omega \nabla\cdot\vec{F}\,{\rm d}\Omega = \int_{d\Omega} \vec{F}\cdot\hat{n}\,{\rm d}S $$

where the RHS is a surface integral. To make the identity we need, we make the substitution $\vec{F}\to v\nabla u$, where later we will write $u\to T$, but its a general identity so we leave it as $u$ for now.

$$ \int_\Omega \nabla\cdot (v\nabla u)\,{\rm d}\Omega = \int_{d\Omega} (v\nabla u)\cdot\hat{n}\,{\rm d}S $$

Expanding the LHS

$$ \int_\Omega (\nabla v) \cdot (\nabla u)\,{\rm d}\Omega + \int_\Omega v\,\nabla\cdot\nabla u\,{\rm d}\Omega = \int_{d\Omega} (v\nabla u)\cdot\hat{n}\,{\rm d}S $$

We note the RHS is a projection of the derivatives onto the normal direction, so we write

$$ \int_\Omega (\nabla v) \cdot (\nabla u)\,{\rm d}\Omega + \int_\Omega v\,\nabla\cdot\nabla u\,{\rm d}\Omega = \int_{d\Omega} v \frac{\partial u}{\partial n}\,{\rm d}S $$

Then we rearrange into the exact form we need

$$ \int_\Omega (\nabla\cdot\nabla u)\,v\,{\rm d}\Omega = \int_{d\Omega} v \frac{\partial u}{\partial n}\,{\rm d}S - \int_\Omega (\nabla v) \cdot (\nabla u)\,{\rm d}\Omega $$

We'll now use this to break out the $\int_{\Omega} (\nabla\cdot k\nabla\vec{T})\,v\,{\rm d}\Omega$ term.

### The final weak form

Using this identity we can split out the fourier term

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega - \int_{\Omega} (\nabla\cdot k\,\nabla\,T)\,v\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega
$$

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega - \int_{d\Omega} v k\frac{\partial T}{\partial n}\,{\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T)\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\vec{r}
$$


### Inserting a Robin boundary condition
Lets use a heat transfer coefficient for the BC

$$
-k\frac{\partial T}{\partial n}=  h (T - T_\infty)
$$


We've only used one boundary condition here for all surfaces, but in general we might get several different terms depending on the BCs defined. Inserting our BC,

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega + \int_{d\Omega} v\,h\,(T - T_\infty){\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T)\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega
$$

Splitting this equation into the linear terms on the RHS (terms without $T$) and bilinear terms on the LHS (terms with $T$) gives,

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega + \int_{d\Omega} v\,h\,T{\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T)\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega +\int_{d\Omega} v\,h\,T_\infty{\rm d}S
$$


## Steady State solution for a heated electrical wire with convection boundary conditions

Lets setup a simulation that solves for the steady state first. Below we create the trial ($u=T$) and test ($v$) functions, then set up the bilinear terms (LHS above, all terms involving the trial function T) and the linear terms (all other terms on RHS). Then we solve.

Here we have heat generation $\sigma=2$ just to get a steady state that is something other than a flat temperature profile.

In [11]:
fes = H1(mesh, order=2)
fes.ndof

T, v = fes.TnT()
k=1
h=1
sigma = 2
Tinf = 5

a = BilinearForm(fes) # This is the LHS of the equation, without the time derivative
a += grad(v) * k * grad(T)*dx
a += v * h * T * ds("outeredge")
a.Assemble()

# Now this is the RHS of the equation
f = LinearForm(sigma * v * dx + v * h * Tinf * ds("outeredge")).Assemble()

# Here we solve it using a matrix inversion
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * f.vec
Draw(gfu)

## General Unsteady state solution of the heat equation
For unsteady state we need to get a weak form of the time integration. Start with the full expression:

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega + \int_{d\Omega} v\,h\,T{\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T)\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega +\int_{d\Omega} v\,h\,T_\infty{\rm d}S
$$

When we define our mesh and solution function space, this weak form will generate a series of equations that need to be solved. This can be written as a set of matrices arising from each term. We will identify the standard matrices that arise from the linear $\vec{f}$ and bilinear $\vec{A}$ terms.

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega + \vec{A}\cdot \vec{T} =  \vec{f}
$$

where the vector $\vec{T}$ is our solution coefficients (not nessacarily the temperature at each grid point, but the coefficients of the functions defined over the grid). We make an implicit Euler approximation of the time derivative as implicit Euler is simple but generally quite stable.

$$
\int_{\Omega} \rho\,C_p\frac{T_{n+1} - T_{n}}{\Delta t} v \,{\rm d}\Omega + \vec{A}\cdot \vec{T}_{n+1} =  \vec{f}
$$

$$
\frac{\rho\,C_p}{\Delta t} \int_{\Omega} \left(T_{n+1} - T_{n}\right) v \,{\rm d}\Omega + \vec{A}\cdot \vec{T}_{n+1} =  \vec{f}
$$


And note that the same operation is happening on the left, so there's one common so-called mass matrix $\vec{M}$ that will result,

$$
\vec{M}\cdot \left(\vec{T}_{n+1} - \vec{T}_{n}\right) + \vec{A}\cdot \vec{T}_{n+1} =  \vec{f}
$$

where,

$$
\vec{M} = \int_{\Omega} \frac{\rho\,C_p}{\Delta t}T\,v \,{\rm d}\Omega
$$

Rearranging,

$$
\left(\vec{M} + \vec{A}\right)\cdot\vec{T}_{n+1} =  \vec{f} + \vec{M}\cdot \vec{T}_{n}
$$

Or defining $\vec{P} = \left(\vec{M} + \vec{A}\right)$,

$$
\vec{P}\cdot\vec{T}_{n+1} =  \vec{f} + \vec{M}\cdot \vec{T}_{n}
$$

The solution is then expressed as the following matrix operations

$$
\vec{T}_{n+1} =  \vec{P}^{-1}\cdot \left(\vec{f} + \vec{M}\cdot \vec{T}_{n}\right)
$$

What is nice is that the matrices $\vec{P}^{-1}$, $\vec{f}$, and $\vec{M}$ do not change, so we can compute these once, then just repeat the above calculation to step forward in time.

# Solution of a initially isothermal cooling rod via convection

In [14]:
# Coefficients for the simulation
rho = 1000 # Density
Cp = 4.18 # Heat capacity
T0 = 100 # Initial temperature
k=10 # Thermal conductivity
h= 1# External heat transfer coefficient
Tinf = 5 # External temperature
R = 1 # Radius of the circle
sigma = 0 # Heat source term
# Simulation duration and time step is defined in terms of tau
tau_end = 5
dtau = tau_end / 100
tend = 10000 # End time
Render_Frames = 100 # Number of frames to render

# Compute external heat transfer Area
pi = 3.14159265359
A_ext = 2 * pi * R # * L
# Compute volume
V = pi * R**2 # * L
# Compute characteristic length
L = V / A_ext
# Compute Biot number
Bi = h * L / k
print(f"Biot number: {Bi}")


tau = rho * Cp * V / h / A_ext
tend = tau_end * tau
dt = dtau * tau

# Compute when to take snapshots
save_every = tend / dt / Render_Frames

#### Build of the geometry and mesh
# Make a circle
geom = Circle((0,0), R).Face()
geom.edges.name = "outeredge"
# Turn it into a 2D mesh, with triangles with a maximum edge length of 0.1
mesh = Mesh(OCCGeometry(geom, dim=2).GenerateMesh(maxh=0.1))

# We now need to define the type of functions that are going to be used over the
# mesh
fes = H1(mesh, order=2)


#### Build of the weak form of the governing equations
A = BilinearForm(fes, symmetric=False) # We need symmetric=False to make sure the sparsity pattern is correct
A += v * h * T * ds("outeredge")
A += grad(v) * k * grad(T)*dx
A.Assemble()

M = BilinearForm(fes, symmetric=False)
M += rho / dt * Cp * T * v * dx
M.Assemble()

# Now this is the RHS of the equation
f = LinearForm(fes)
f += sigma*v*dx
f += v * h * Tinf * ds("outeredge")
f.Assemble()

P = M.mat.CreateMatrix()
P.AsVector().data = M.mat.AsVector() + A.mat.AsVector()
invP = P.Inverse(freedofs=fes.FreeDofs())

#### Time integration
# Prepare the time integration starting point
time = 0
step_idx = 0
# T_last is the current solution to the temperature field
T_last = GridFunction(fes)
T_last.Set(T0) # Set the initial condition
# Prepare the live visualization
scene = Draw(T_last, mesh, "T", min=Tinf, max=T0)
# Store the history of the temperature for visualization
T_history = GridFunction(T_last.space, multidim=0)
T_history.AddMultiDimComponent(T_last.vec)
# Time integration loop
T_avg_list = []
while time < tend - dt/2:
    # Test if we need to save the current state
    if step_idx % save_every == 0:
        T_history.AddMultiDimComponent(T_last.vec)

    # Compute the average temperature
    T_avg = Integrate(T_last, mesh) / V
    T_avg_list.append((time, T_avg))
    # Perform the time integration
    T_last.vec.data = invP * (f.vec + M.mat * T_last.vec)
    # Redraw the visualization
    scene.Redraw()

    # Update the time and step index
    time += dt
    step_idx += 1

# Save animation here
# https://docu.ngsolve.org/latest/i-tutorials/appendix-vtk/vtk.html

from matplotlib import pyplot as plt
%matplotlib inline
time = [t for t, T in T_avg_list]
T_sim = [T for t, T in T_avg_list]
plt.plot(time, T_sim, label="Simulation")
#Draw(T_history, mesh, interpolate_multidim=False, min=Tinf, max=T0, animate=True, speed=10)
T_lumped = [Tinf + (T0 - Tinf) * exp(-t / tau) for t in time]
plt.plot(time, T_lumped, label="Lumped")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Average temperature (°C)")
plt.title(f"Bi={Bi}")

Biot number: 0.05


AttributeError: 'NoneType' object has no attribute 'Redraw'

## Appendix: Doing it without the matrix math

The first time I solved this, I left the time-stepping in the weak form construction. It works but is inefficient. Start with the full expression:

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\Omega + \int_{d\Omega} v\,h\,T{\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T)\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega +\int_{d\Omega} v\,h\,T_\infty{\rm d}S
$$

We replace the time derivative with an implicit Euler, which means that all
temperatures are now evalulated in the next time step. Implicit Euler is chosen
as its generally quite stable compared to other schemes, but needs a full
inversion to work.

$$
\int_{\Omega} \rho\,C_p\frac{\left(T_{n+1} - T_{n}\right)}{\Delta t} v \,{\rm d}\Omega + \int_{d\Omega} v\,h\,T_{n+1}{\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T_{n+1})\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega +\int_{d\Omega} v\,h\,T_\infty{\rm d}S
$$

The term involving the previous time step is now no-longer a function of the current trial function $T_{n+1}$ so we move it over to the linear side.

$$
\int_{\Omega} \rho\,C_p\frac{T_{n+1}}{\Delta t} v \,{\rm d}\Omega + \int_{d\Omega} v\,h\,T_{n+1}{\rm d}S + \int_\Omega (\nabla v) \cdot (k\nabla T_{n+1})\,{\rm d}\Omega =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\Omega +\int_{d\Omega} v\,h\,T_\infty{\rm d}S + \int_{\Omega} \rho\,C_p\frac{T_{n}}{\Delta t} v \,{\rm d}\Omega
$$


$$
\int_{\Omega} \left[\rho\,C_p\frac{T_{n+1}}{\Delta t} v+(\nabla v) \cdot (k\nabla T_{n+1})\right]\,{\rm d}\Omega  + \int_{d\Omega} v\,h\,T_{n+1}\,{\rm d}S =  \int_{\Omega}\left[\sigma_{energy}\,v +\rho\,C_p\frac{T_{n}}{\Delta t} v\right]{\rm d}\Omega +\int_{d\Omega} v\,h\,T_\infty{\rm d}S
$$


In [ ]:
# This is an example of taking one step in time the "proper way"
T_last = GridFunction(fes)
T_last.Set(100)

rho = 1000
Cp = 4.18
dt = 0.1

a = BilinearForm(fes) # This is the LHS of the equation, WITH time derivatives
a += (rho / dt * Cp) * T * v * dx
a += grad(v) * k * grad(T)*dx
a += v * h * T * ds("outeredge")
a.Assemble()

# Now this is the RHS of the equation
f = LinearForm(fes)
f += sigma*v*dx
f += v * h * Tinf * ds("outeredge")
f += rho * Cp / dt * T_last * v * dx
f.Assemble()

# Here we solve it using a matrix inversion
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * f.vec

scene = Draw(gfu, mesh, "T")